---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [91]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [92]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [93]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    states_list = []
    states_dict = {}
    state_amount = {}
    states_multi = []
    
    df = pd.read_table("university_towns.txt", header=None, names=["RegionName"])
    
    for name in df["RegionName"]:
        if "[edit]" in name:
            idx = df.RegionName[df.RegionName == name].index.tolist()[0]
            states_dict[name] = idx
            name = name.replace("[edit]", "")
            states_list.append(name)
            
    states_list_raw = states_dict.keys()
    indexes = list(states_dict.values())
    index_pairs = [tuple(indexes[i:i+2]) for i in range(0, len(indexes))]
    diff = [idx_pair[-1] - idx_pair[0] for idx_pair in index_pairs]
    diff[-1] = len(df) - index_pairs[-1][-1]
    
    for i in range(len(states_list)):
        state_amount[states_list[i]] = diff[i]
        
    for state_name, length in state_amount.items():
        states_multi += [state_name] * length
                
    df["State"] = states_multi 
    df = df[~df["RegionName"].isin(states_list_raw)]
    df["RegionName"] = df["RegionName"].str.replace(r"\(.*\)","")
    df["RegionName"] = df["RegionName"].str.replace(r"\[.*\]","")
    df["RegionName"] = df["RegionName"].str.replace(r"[\s,.]","")
        
    return df.reset_index().drop("index", axis=1)
get_list_of_university_towns()

,RegionName,State
0,Auburn,Alabama
1,Florence,Alabama
2,Jacksonville,Alabama
3,Livingston,Alabama
4,Montevallo,Alabama
5,Troy,Alabama
6,Tuscaloosa,Alabama
7,Tuskegee,Alabama
8,Fairbanks,Alaska
9,Flagstaff,Arizona


In [94]:
df = pd.read_excel("gdplev.xls", header=5, usecols = [0, 1, 2, 4, 5, 6], names=["Year", "GDP in billions of current dollars", "GDP in billions of chained 2009 dollars", "Quarter", "gdp", "GDP in billions of chained 2009 dollars.1"]).drop([0, 1])
quarter_gdp = df.iloc[:, [3,4]].reset_index().drop("index", axis=1)

recession_start = []
index_start = []

for i in range(len(quarter_gdp) - 1):
    if quarter_gdp.loc[i, "gdp"] > quarter_gdp.loc[i+1, "gdp"]:
        recession_start.append(quarter_gdp.iloc[i, 0])
        index_start.append(i)

for i in range((len(quarter_gdp) - 1), -1, -1):
    if quarter_gdp.loc[i, "Quarter"] in recession_start and quarter_gdp.loc[i-1, "Quarter"] in recession_start:
        recession_start.remove(quarter_gdp.loc[i, "Quarter"])
        index_start.remove(i)
        
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    return recession_start
get_recession_start()

['1948q4',
 '1949q3',
 '1953q2',
 '1957q3',
 '1960q1',
 '1960q3',
 '1981q4',
 '1990q3',
 '2007q4',
 '2008q3']

In [95]:
recession_end = []
index_end = []

for i in range(len(quarter_gdp)-1):
    for start in recession_start:
        if start == quarter_gdp.loc[i, "Quarter"]:
            for n in range(0, (len(quarter_gdp)-1)):
                if quarter_gdp.loc[i, "gdp"] < quarter_gdp.loc[i+n, "gdp"]:
                    recession_end.append(quarter_gdp.loc[i+n, "Quarter"])
                    index_end.append(i+n)
                    break

def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    return recession_end

get_recession_end()

['1950q1',
 '1950q1',
 '1954q4',
 '1958q3',
 '1960q3',
 '1961q2',
 '1982q2',
 '1991q1',
 '2008q2',
 '2010q2']

In [96]:
just_gdp = quarter_gdp.loc[:, "gdp"]
index_tuples = list(map(lambda x, y:(x,y), index_start, index_end)) 
index_bottom = [just_gdp[pair[0]:pair[1]].idxmin() for pair in index_tuples]
recession_bottom = [quarter_gdp.loc[index, "Quarter"] for index in index_bottom]
    
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    return recession_bottom

get_recession_bottom()

['1949q4',
 '1949q4',
 '1954q1',
 '1958q1',
 '1960q2',
 '1960q4',
 '1982q1',
 '1990q4',
 '2008q1',
 '2009q2']

In [97]:

def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    quarter_list = []
    
    housing = pd.read_csv("City_Zhvi_AllHomes.csv")
    housing = housing.loc[:,housing.columns.str.contains("State|RegionName|20")]
    housing = housing.set_index(["State", "RegionName"])
    
    for i in range(0, 17):
        for n in range(1, 5):
            if i == 16 and n==4:
                break
            elif i<10:
                quarter_list.append('200{}q{}'.format(i, n))
            else:
                quarter_list.append('20{}q{}'.format(i, n))
                
    tripled_quarters = [val for trio in zip(quarter_list, quarter_list, quarter_list) for val in trio]
    del tripled_quarters[-1]
    housing.columns = tripled_quarters
    housing = housing.groupby(housing.columns, axis=1).mean()
    
    return housing
convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
NY,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.154667e+05,5.228000e+05,5.280667e+05,5.322667e+05,5.408000e+05,5.572000e+05,5.728333e+05,5.828667e+05,5.916333e+05,587200.0
CA,Los Angeles,2.070667e+05,2.144667e+05,2.209667e+05,2.261667e+05,2.330000e+05,2.391000e+05,2.450667e+05,2.530333e+05,2.619667e+05,2.727000e+05,...,4.980333e+05,5.090667e+05,5.188667e+05,5.288000e+05,5.381667e+05,5.472667e+05,5.577333e+05,5.660333e+05,5.774667e+05,584050.0
IL,Chicago,1.384000e+05,1.436333e+05,1.478667e+05,1.521333e+05,1.569333e+05,1.618000e+05,1.664000e+05,1.704333e+05,1.755000e+05,1.775667e+05,...,1.926333e+05,1.957667e+05,2.012667e+05,2.010667e+05,2.060333e+05,2.083000e+05,2.079000e+05,2.060667e+05,2.082000e+05,212000.0
PA,Philadelphia,5.300000e+04,5.363333e+04,5.413333e+04,5.470000e+04,5.533333e+04,5.553333e+04,5.626667e+04,5.753333e+04,5.913333e+04,6.073333e+04,...,1.137333e+05,1.153000e+05,1.156667e+05,1.162000e+05,1.179667e+05,1.212333e+05,1.222000e+05,1.234333e+05,1.269333e+05,128700.0
AZ,Phoenix,1.118333e+05,1.143667e+05,1.160000e+05,1.174000e+05,1.196000e+05,1.215667e+05,1.227000e+05,1.243000e+05,1.265333e+05,1.283667e+05,...,1.642667e+05,1.653667e+05,1.685000e+05,1.715333e+05,1.741667e+05,1.790667e+05,1.838333e+05,1.879000e+05,1.914333e+05,195200.0
NV,Las Vegas,1.326000e+05,1.343667e+05,1.354000e+05,1.370000e+05,1.395333e+05,1.417333e+05,1.433667e+05,1.461333e+05,1.493333e+05,1.509333e+05,...,1.700667e+05,1.734000e+05,1.754667e+05,1.775000e+05,1.816000e+05,1.867667e+05,1.906333e+05,1.946000e+05,1.972000e+05,199950.0
CA,San Diego,2.229000e+05,2.343667e+05,2.454333e+05,2.560333e+05,2.672000e+05,2.762667e+05,2.845000e+05,2.919333e+05,3.012333e+05,3.128667e+05,...,4.802000e+05,4.890333e+05,4.964333e+05,5.033667e+05,5.120667e+05,5.197667e+05,5.254667e+05,5.293333e+05,5.362333e+05,539750.0
TX,Dallas,8.446667e+04,8.386667e+04,8.486667e+04,8.783333e+04,8.973333e+04,8.930000e+04,8.906667e+04,9.090000e+04,9.256667e+04,9.380000e+04,...,1.066333e+05,1.089000e+05,1.115333e+05,1.137000e+05,1.211333e+05,1.285667e+05,1.346000e+05,1.405000e+05,1.446000e+05,149300.0
CA,San Jose,3.742667e+05,4.065667e+05,4.318667e+05,4.555000e+05,4.706667e+05,4.702000e+05,4.568000e+05,4.455667e+05,4.414333e+05,4.577667e+05,...,6.794000e+05,6.970333e+05,7.149333e+05,7.314333e+05,7.567333e+05,7.764000e+05,7.891333e+05,8.036000e+05,8.189333e+05,822200.0


In [101]:
# For some reason this function takes 4 notebook compilations to work properly (KeyError occurrs for the first 3 times)
# TODO: Investigate and fix the problem described above.

def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    housing = convert_housing_data_to_quarters()
    uni_towns_df = get_list_of_university_towns()
    list_uni_towns = uni_towns_df.RegionName.to_list()
    
    for start in recession_start:
        if start not in housing.columns:
            recession_start.remove(start)
    for bottom in recession_bottom:
        if bottom not in housing.columns:
            recession_bottom.remove(bottom)
    start_bottom_pair = list(map(lambda x, y: (x, y), recession_start, recession_bottom))

    for i in range(len(start_bottom_pair)):
        df_slice = housing.loc[:, start_bottom_pair[i][0]] / housing.loc[:, start_bottom_pair[i][1]]
        if i == 0:
            diff = pd.DataFrame(df_slice).rename(columns={0: "1st recession"})
        else:
            diff["2nd recession"] = df_slice
            
    diff = diff.apply(lambda x: x["1st recession"] + x["2nd recession"], axis=1).reset_index().replace({"State": states}).rename(columns={0: "change"})
    
    for i in range(len(diff)):
        if diff.loc[i, "RegionName"] in list_uni_towns:
            diff.at[i, "RegionName"] = "university town"
        else:
            diff.at[i, "RegionName"] = "non-university town"
            
    diff_mean = diff.groupby("RegionName").mean()
    
    if diff_mean.loc["non-university town", "change"] > diff_mean.loc["university town", "change"]:
        better = "university town"
    else:
        better = "non-university town"
    
    non_uni = diff.where(diff.RegionName == "non-university town").dropna()["change"]
    uni = diff.where(diff.RegionName == "university town").dropna()["change"]
    ttest = ttest_ind(non_uni, uni)
    pvalue = ttest[-1]
    different = pvalue < 0.01
    
    return (different, pvalue, better)
run_ttest()

(True, 7.478998700605076e-05, 'university town')